In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import urlopen
import requests
import sqlite3
import json
import sqlite3
import os


In [2]:
#GUARDANDO LISTAS
Urls_Carrefour = []
Sellers_Carrefour = []
Country_Carrefour = []
Price_Carrefour = []
Price_Carrefour_2 = []
SKU_Carrefour = []
Title_Carrefour = []
Installment_Carrefour_quantidade = []
Installment_Carrefour_valor_parcela = []
Installment_Carrefour_valor_total = []

In [3]:
header = {'authority':'www.carrefour.com.br', 'scheme':'https', 'accept':'application/json','user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}


In [4]:
def getting_n_creating_carrefour_urls(brand):
    # Pegando caminho do database
    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "%20")
    df['Product_Name'] = df['Product_Name'].str.replace("-", "%20")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "%20" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.carrefour.com.br/busca/" + df['Urls']

    return df

In [5]:
Df_products = getting_n_creating_carrefour_urls("Wacom")
Df_products

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom%20ctl472,https://www.carrefour.com.br/busca/wacom%20ctl472
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom%20ctl4100,https://www.carrefour.com.br/busca/wacom%20ctl...
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom%20ctl4100wle0,https://www.carrefour.com.br/busca/wacom%20ctl...
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom%20ctl4100wlk0,https://www.carrefour.com.br/busca/wacom%20ctl...
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom%20ctl6100wlk0,https://www.carrefour.com.br/busca/wacom%20ctl...
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom%20dtc133w0a,https://www.carrefour.com.br/busca/wacom%20dtc...
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom%20pth460,https://www.carrefour.com.br/busca/wacom%20pth460
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom%20dtk1660k0a,https://www.carrefour.com.br/busca/wacom%20dtk...
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom%20pth660,https://www.carrefour.com.br/busca/wacom%20pth660
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom%20ctl6100wle0,https://www.carrefour.com.br/busca/wacom%20ctl...


In [6]:
def get_atributes(url):
    time.sleep(5)

    response = requests.get(url, headers=header)
    html = response.text

    bs = BeautifulSoup(html, 'html.parser')

    template = bs.find('template', attrs={'data-type':'json','data-varname':'__STATE__'})

    text = template.contents[1].string

    data = json.loads(text)

    principal_key = list(data.keys())

    keys_principal = []

    for key in principal_key:
        keys_principal.append(key)

    keys_principal = [s for s in keys_principal if 'Product' in s]
    keys_principal = [s for s in keys_principal if not '$' in s]

    Data_json = pd.DataFrame()
    Data_json['Keys'] = keys_principal
    Data_json['Code'] = Data_json['Keys'].str.partition(":")[2].str.partition(".")[0]
    Data_json = Data_json.drop_duplicates(subset=['Code'])
    Data_json['Price_id'] = "$" + Data_json['Keys'] + '.items({\"filter\":\"ALL_AVAILABLE\"}).0.sellers.0.commertialOffer'
    Data_json['Sellers_id'] = Data_json['Keys'] + '.items({\"filter\":\"ALL_AVAILABLE\"}).0.sellers.0'

    #Pegando title
    for key in Data_json['Keys']:
        try:
            Title_Carrefour.append(data[key]['productName'])
        except:
            Title_Carrefour.append("Erro")

    #Pegando SKU
    for key in Data_json['Keys']:
        try:
            SKU_Carrefour.append(data[key]['productReference'])
        except:
            SKU_Carrefour.append("Erro")

    #Pegando Url
    for key in Data_json['Keys']:
        try:
            Urls_Carrefour.append("www.carrefour.com.br" + data[key]['link'])
        except:
            Urls_Carrefour.append("Erro")

    #Pegando Price
    for key in Data_json['Price_id']:
        try:
            Price_Carrefour.append(data[key]['Price'])
        except:
            Price_Carrefour.append("Erro")

    #Pegando Price 2
    for key in Data_json['Price_id']:
        try:
            Price_Carrefour_2.append(data[key]['spotPrice'])
        except:
            Price_Carrefour_2.append("Erro")

    #Pegando Seller
    for key in Data_json['Sellers_id']:
        try:
            Sellers_Carrefour.append(data[key]['sellerName'])
        except:
            Sellers_Carrefour.append("Erro")

In [7]:
for url in tqdm(Df_products['Urls_search']):
    get_atributes(url)

100%|██████████| 10/10 [01:20<00:00,  8.07s/it]


In [8]:
def creating_dataframe(urls, loja, title, price, price_2, sellers, sku):
    Dataset = pd.DataFrame()

    Dataset['Urls'] = urls
    Dataset['Loja'] = loja
    Dataset['Title'] = title
    Dataset['Price'] = price
    Dataset['Price_2'] = price_2
    Dataset['Sellers'] = sellers
    Dataset['SKU'] = sku

    return Dataset

In [9]:
Dataset_Carrefour = creating_dataframe(Urls_Carrefour,"CARREFOUR",Title_Carrefour,Price_Carrefour,Price_Carrefour_2,Sellers_Carrefour,SKU_Carrefour)

In [10]:
Dataset_Carrefour

,Urls,Loja,Title,Price,Price_2,Sellers,SKU
0,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom By One - Pequena - C...,306.99,306.99,Oficina dos Bits,MP21128371W-MV21128370
1,www.carrefour.com.br/mesa-digitalizadora-one-b...,CARREFOUR,Mesa Digitalizadora One By Wacom Ctl-472 Pequena,299.00,299.00,ITX GAMER,MP924300034
2,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom Ctl472 One,0.00,0.00,ALLIANCEINFORMATICA COMERCIO ELETRONICO,MP922297075
3,www.carrefour.com.br/mesa-digitalizadora-one-b...,CARREFOUR,Mesa Digitalizadora One By Wacom Ctl472,578.78,578.78,Olist Store,MP922297033
4,www.carrefour.com.br/mesa-digitalizadora-one-b...,CARREFOUR,Mesa Digitalizadora One by Wacom CTL472 Pequena,299.90,299.90,Primetek,MP909230271
...,...,...,...,...,...,...,...
62,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom Intuos Pro Medium Bl...,0.00,0.00,Carrefour,MP29125357
63,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom CTL-6100WL 4096 Pont...,1290.00,1290.00,INPOWER,MP19610271
64,www.carrefour.com.br/mesadigitalizadorawacomct...,CARREFOUR,Mesa Digitalizadora Wacom Ctl6100wle0 Intuos M...,0.00,0.00,ALLIANCEINFORMATICA COMERCIO ELETRONICO,MP922638005
65,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom Intuos Media com Blu...,0.00,0.00,Trade Hub,MP14322562


In [11]:
Dataset_Carrefour.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Carrefour_Wacom.xlsx", index=False)